In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime
import yaml

import gymnasium as gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

from env import SlidingEnv
import wrappers

pygame 2.5.2 (SDL 2.28.2, Python 3.8.17)
Hello from the pygame community. https://www.pygame.org/contribute.html


/home/bryan/miniconda3/envs/dreamer/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/home/bryan/miniconda3/envs/dreamer/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):
2023-11-27 19:08:32.485436: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-27 19:08:32.965312: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/bryan/miniconda3/envs/dreamer/lib/python3.8/site-packages/pandas/compat/nump

In [8]:
configs = {
    "run_id": datetime.datetime.now().strftime('%Y%m%d-%H%M%S'),
    "policy": "MlpPolicy", # MlpPolicy or CnnPolicy
    "total_timesteps": 1000000,

    "env_kwargs": {"w": 2, "h": 2, "shuffle_steps": 5, "sparse_rewards": True},
    "n_envs": 32,
    # "wrapper_class": wrappers.OneHotEncodingWrapper,
    "wrapper_class": wrappers.NormalizedObsWrapper,

    "max_episode_steps": 100,
    "seed": 42,
}

In [9]:
gym.envs.register(
    id="SlidingEnv-v0",
    entry_point=SlidingEnv,
    max_episode_steps=configs["max_episode_steps"],
)

env = make_vec_env(
    env_id="SlidingEnv-v0",
    seed=configs["seed"],
    wrapper_class=configs["wrapper_class"],
    env_kwargs=configs["env_kwargs"],
    n_envs=configs["n_envs"],
    monitor_dir=f"runs/{datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}/logs",
    monitor_kwargs={"info_keywords": ("is_success",)},
)

/home/bryan/miniconda3/envs/dreamer/lib/python3.8/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment SlidingEnv-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [10]:
model = PPO(
    configs["policy"],
    env,
    verbose=2,
    tensorboard_log=f"runs/{configs['run_id']}",
    n_steps=configs["max_episode_steps"],
)

Using cuda device


In [11]:
from stable_baselines3.common.callbacks import BaseCallback

class SuccessRateCallback(BaseCallback):
    def __init__(self, verbose=0):
        super(SuccessRateCallback, self).__init__(verbose)

    def _on_step(self) -> bool:
        return True

    def _on_rollout_end(self) -> None:
        episodes = len(self.locals["infos"])
        
        if episodes > 0:
            successes = 0
            for i_info in self.locals["infos"]:
                if i_info.get("is_success"):
                    successes += 1

            success_rate = successes / episodes
            self.logger.record("rollout/success_rate", success_rate)

model.learn(
    total_timesteps=configs["total_timesteps"],
    progress_bar=True,
    callback=SuccessRateCallback(),
)

with open(f"runs/{configs['run_id']}/configs.yaml", "w") as f:
    yaml.dump(configs, f)
model.save(f"runs/{configs['run_id']}/model")

Logging to runs/20231127-192539/PPO_1


Output()

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 58.2     |
|    ep_rew_mean     | -20.7    |
|    success_rate    | 0        |
| time/              |          |
|    fps             | 18059    |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 3200     |
---------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 53.3        |
|    ep_rew_mean          | -17.4       |
|    success_rate         | 0           |
| time/                   |             |
|    fps                  | 3314        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 6400        |
| train/                  |             |
|    approx_kl            | 0.011861847 |
|    clip_fraction        | 0.152       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.38       |
|    explained_variance   | 0.00839     |
|    learning_rate        | 0.0003      |
|    loss                 | 9.49        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0125     |
|    value_loss           | 21.8        |
-----------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 50.4        |
|    ep_rew_mean          | -13.7       |
|    success_rate         | 0.0312      |
| time/                   |             |
|    fps                  | 2482        |
|    iterations           | 3           |
|    time_elapsed         | 3           |
|    total_timesteps      | 9600        |
| train/                  |             |
|    approx_kl            | 0.010448278 |
|    clip_fraction        | 0.139       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.35       |
|    explained_variance   | 0.0296      |
|    learning_rate        | 0.0003      |
|    loss                 | 17.2        |
|    n_updates            | 20          |
|    policy_gradient_loss | -0.0133     |
|    value_loss           | 32.2        |
-----------------------------------------


...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 6.1         |
|    ep_rew_mean          | 9.96        |
|    success_rate         | 0.0625      |
| time/                   |             |
|    fps                  | 1729        |
|    iterations           | 309         |
|    time_elapsed         | 571         |
|    total_timesteps      | 988800      |
| train/                  |             |
|    approx_kl            | 0.033301815 |
|    clip_fraction        | 0.124       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.0468     |
|    explained_variance   | 0.5         |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0182     |
|    n_updates            | 3080        |
|    policy_gradient_loss | -0.0339     |
|    value_loss           | 0.0821      |
-----------------------------------------


----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 14.2       |
|    ep_rew_mean          | 8.44       |
|    success_rate         | 0          |
| time/                   |            |
|    fps                  | 1730       |
|    iterations           | 310        |
|    time_elapsed         | 573        |
|    total_timesteps      | 992000     |
| train/                  |            |
|    approx_kl            | 0.25506604 |
|    clip_fraction        | 0.3        |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.18      |
|    explained_variance   | 0.515      |
|    learning_rate        | 0.0003     |
|    loss                 | -0.0513    |
|    n_updates            | 3090       |
|    policy_gradient_loss | 0.111      |
|    value_loss           | 0.0104     |
----------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 9.58        |
|    ep_rew_mean          | 9.52        |
|    success_rate         | 0.0938      |
| time/                   |             |
|    fps                  | 1731        |
|    iterations           | 311         |
|    time_elapsed         | 574         |
|    total_timesteps      | 995200      |
| train/                  |             |
|    approx_kl            | 0.041198403 |
|    clip_fraction        | 0.308       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.349      |
|    explained_variance   | 0.0491      |
|    learning_rate        | 0.0003      |
|    loss                 | 0.539       |
|    n_updates            | 3100        |
|    policy_gradient_loss | -0.0453     |
|    value_loss           | 1.34        |
-----------------------------------------


----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 8.08       |
|    ep_rew_mean          | 10         |
|    success_rate         | 0.125      |
| time/                   |            |
|    fps                  | 1732       |
|    iterations           | 312        |
|    time_elapsed         | 576        |
|    total_timesteps      | 998400     |
| train/                  |            |
|    approx_kl            | 0.20017043 |
|    clip_fraction        | 0.28       |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.212     |
|    explained_variance   | 0.232      |
|    learning_rate        | 0.0003     |
|    loss                 | 0.352      |
|    n_updates            | 3110       |
|    policy_gradient_loss | -0.0561    |
|    value_loss           | 0.641      |
----------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 6.74        |
|    ep_rew_mean          | 10          |
|    success_rate         | 0.25        |
| time/                   |             |
|    fps                  | 1733        |
|    iterations           | 313         |
|    time_elapsed         | 577         |
|    total_timesteps      | 1001600     |
| train/                  |             |
|    approx_kl            | 0.007582371 |
|    clip_fraction        | 0.231       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.171      |
|    explained_variance   | 0.293       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0227      |
|    n_updates            | 3120        |
|    policy_gradient_loss | -0.0356     |
|    value_loss           | 0.092       |
-----------------------------------------
